In [ ]:
!pip install \
    alifedata_phyloinformatics_convert==0.16.2 \
    colorclade \
    dendropy \
    hstrat==1.11.7 \
    "pandas<2" \
    "pyarrow" \
    phylotrackpy==0.2.0 \
    tqdm


In [ ]:
import alifedata_phyloinformatics_convert as apc
from colorclade import draw_colorclade_tree
import dendropy as dp
from hstrat import _auxiliary_lib as hstrat_aux
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [ ]:
df = pd.read_parquet("https://osf.io/r2a7t/download")

Subset to interesting data.

In [ ]:
df = df[
    (df["nCycle"] == df["nCycle"].max())
    & (df["popSize"] == df["popSize"].max())
].reset_index(drop=True)

- purifying only: only deleterious mutations allowed
- purifying plus: both deleterious and beneficial allowed

In [ ]:
df["genomeFlavor"].unique()

## Visualize

In [ ]:
def sample_n_leaves(df: pd.DataFrame, n: int=10) -> pd.DataFrame:
    df = df.copy()
    df["extant"] = df["id"].isin(
        hstrat_aux.alifestd_find_leaf_ids(df.sample(frac=1.0))[:n],
    )
    return hstrat_aux.alifestd_prune_extinct_lineages_asexual(df)


In [ ]:
def log_scale_origin_time(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = hstrat_aux.alifestd_mark_leaves(df)
    diff = np.ptp(df.loc[df["is_leaf"], "origin_time"])
    df["origin_time"] = np.log(df["origin_time"].max() + diff) - np.log(df["origin_time"].max() - df["origin_time"] + diff)
    return df


In [ ]:
ncol = 4
nrow = (df["replicate"].nunique() - 1) // ncol + 1
fig, axs = plt.subplots(nrow, ncol)

for ax, ((flavor, replicate), group) in tqdm(list(zip(
    axs.flat, df.groupby(["genomeFlavor", "replicate"])
))):
    draw_colorclade_tree(
        log_scale_origin_time(sample_n_leaves(group, 20)),
        taxon_name_key="id",
        backend="biopython",
        ax=ax,
        label_tips=False,
    )
    newline = "\n"
    ax.set_title(f"{flavor.replace('_', newline)}")
    ax.set_xlabel("")
    print(f"{flavor=} {replicate=}")

plt.tight_layout()

## Calculate Phylometrics

In [ ]:
records = []
for (flavor, replicate), group in tqdm(
    df.groupby(["genomeFlavor", "replicate"]),
):
    syst = apc.RosettaTree(
        group,
    ).as_phylotrack

    records.append(
        {
            "genomeFlavor": flavor,
            "replicate": replicate,
            "colless-like index": syst.colless_like_index(),
            "evo distinctness": syst.get_mean_evolutionary_distinctiveness(
                group["origin_time"].max(),
            ),
            "sum branch length": syst.get_sum_distance(),
        }
    )


In [ ]:
sns.catplot(
    data=pd.melt(
        pd.DataFrame.from_records(records),
        id_vars=["genomeFlavor", "replicate"],
        value_vars=[
            "colless-like index",
            "evo distinctness",
            "sum branch length",
        ],
        var_name="metric",
    ),
    x="genomeFlavor",
    y="value",
    col="metric",
    hue="genomeFlavor",
    sharey=False,
    col_wrap=2,
    height=2,
    aspect=2,
)